<a href="https://colab.research.google.com/github/MiM0ulay/RiskMetrics/blob/main/RiskMetricFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nasdaq-data-link


In [2]:
from datetime import date
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import nasdaqdatalink
import yfinance as yf

In [3]:
# Download historical data from Quandl
df = nasdaqdatalink.get_table("QDL/BCHAIN", api_key='rRQgKH5b9t6Bmb1z81QE', paginate=True)
df = df[df["code"] == "MKPRU"].reset_index()

In [4]:
# Convert dates to datetime object for easy use
df['date'] = pd.to_datetime(df['date'])

# Sort data by date, just in case
df.sort_values(by='date', inplace=True)

In [5]:
# Only include data points with existing price
df = df[df['value'] > 0]

In [6]:
# get data thats not in the quandl database
new_data = yf.download(tickers='BTC-USD', start='2024-01-01', interval='1d')
new_data.reset_index(inplace=True)
# restructure yf dataframe to match the quandl one
new_data.rename(columns={'Date': 'date', 'Open': 'value'}, inplace=True)
new_data = new_data[['date', 'value']]
df = pd.concat([df, new_data], ignore_index=True)
df.drop_duplicates(subset='date', keep='first', inplace=True)
df.sort_values(by='date', inplace=True)


[*********************100%***********************]  1 of 1 completed


In [7]:
btcdata = yf.download(tickers='BTC-USD', period='1d', interval='1m')
btcdata.reset_index(inplace=True)
#btcdata.rename(columns={'Datetime': 'date', 'Open': 'value'}, inplace=True)
#df.loc[df.index[-1]+1] = [date.today(), btcdata['Close'].iloc[-1]]

[*********************100%***********************]  1 of 1 completed


In [8]:

df = df.dropna(subset=['value'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5237 entries, 0 to 5236
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   None              5237 non-null   float64       
 1   code              5237 non-null   object        
 2   date              5237 non-null   datetime64[ns]
 3   value             5237 non-null   float64       
 4   (date, )          0 non-null      datetime64[ns]
 5   (value, BTC-USD)  0 non-null      float64       
dtypes: datetime64[ns](2), float64(3), object(1)
memory usage: 286.4+ KB
None


In [9]:
df = df[['date', 'value']]

In [10]:
diminishing_factor = 0.395
moving_average_days = 365

In [11]:
# Calculate the `Risk Metric`
    # calculate the x day moving average
df['MA'] = df['value'].rolling(moving_average_days, min_periods=1).mean().dropna()
    # calculate log-return adjusted to diminishing returns over time
    # this log-return is the relative price change from the moving average

In [12]:
df['Preavg'] = (np.log(df.value) - np.log(df['MA'])) * df.index**diminishing_factor

In [13]:
# Normalization to 0-1 range
df['avg'] = (df['Preavg'] - df['Preavg'].cummin()) / (df['Preavg'].cummax() - df['Preavg'].cummin())

In [14]:
price_per_risk = {
    round(risk, 1):round(np.exp(
        (risk * (df['Preavg'].cummax().iloc[-1] - (cummin := df['Preavg'].cummin().iloc[-1])) + cummin) / df.index[-1]**diminishing_factor + np.log(df['MA'].iloc[-1])
    ))
    for risk in np.arange(0.0, 1.0, 0.1)
}

In [15]:
# # Exclude the first 1000 days from the dataframe, because it's pure chaos
#df = df[df.index > 1000]

In [16]:
AnnotationText = f"Updated: {btcdata.index[-1]} | Price: {round(df['value'].iloc[-1])} | Risk: {round(df['avg'].iloc[-1], 2)}"

In [17]:
# Plot BTC-USD and Risk on a logarithmic chart
fig = make_subplots(specs=[[{'secondary_y': True}]])

# Add BTC-USD and Risk data to the figure
fig.add_trace(go.Scatter(x=df['date'], y=df['value'], name='Price', line=dict(color='gold')))
fig.add_trace(go.Scatter(x=df['date'], y=df['avg'],   name='Risk',  line=dict(color='white')), secondary_y=True)

# Add green (`accumulation` or `buy`) rectangles to the figure
opacity = 0.2
for i in range(5, 0, -1):
    opacity += 0.05
    fig.add_hrect(y0=i*0.1, y1=((i-1)*0.1), line_width=0, fillcolor='green', opacity=opacity, secondary_y=True)

# Add red (`distribution` or `sell`) rectangles to the figure
opacity = 0.2
for i in range(6, 10):
    opacity += 0.1
    fig.add_hrect(y0=i*0.1, y1=((i+1)*0.1), line_width=0, fillcolor='red', opacity=opacity, secondary_y=True)

fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_yaxes(title='Risk', type='linear', secondary_y=True, showgrid=True, tick0=0.0, dtick=0.1, range=[0, 1])
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot BTC-USD colored according to Risk values on a logarithmic chart
fig = px.scatter(df, x='date', y='value', color='avg', color_continuous_scale='jet')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot Predicting BTC price according to specific risk
fig = go.Figure(data=[go.Table(
    header=dict(values=['Risk', 'Price'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[list(price_per_risk.keys()), list(price_per_risk.values())],
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])
fig.update_layout(width=500, height=500, title={'text': 'Price according to specific risk', 'y': 0.9, 'x': 0.5})
fig.show()

In [18]:
df.tail(60)

,date,value,MA,Preavg,avg
5177,2024-10-24,66655.68,56000.454466,5.105759,0.445128
5178,2024-10-25,68162.43,56105.878658,5.706305,0.454783
5179,2024-10-26,66639.48,56206.475370,4.991753,0.443295
5180,2024-10-27,67018.78,56307.922082,5.105672,0.445127
5181,2024-10-28,67938.81,56403.480521,5.456135,0.450761
5182,2024-10-29,69913.46,56502.140274,6.245463,0.463451
5183,2024-10-30,72706.28,56606.833671,7.340355,0.481053
5184,2024-10-31,72329.87,56711.414712,7.134546,0.477744
5185,2024-11-01,70219.02,56810.888521,6.214965,0.462960
5186,2024-11-02,69490.73,56907.874959,5.859581,0.457247
